In [6]:
import pandas as pd
import numpy as np
import tmdbsimple as tmdb
import json, os, time
from tqdm.notebook import tqdm_notebook

In [9]:
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['title.akas.tsv.gz', 'title.basics.tsv.gz', 'title.ratings.tsv.gz']

In [12]:
#assign data to variables
title_basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
title_akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
title_ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [14]:
# Open saved file and preview again
basics = pd.read_csv(title_basics_url,sep='\t', low_memory=False)

In [15]:
#read variable to dataframe
akas = pd.read_csv(title_akas_url,sep='\t', low_memory=False)

In [18]:
rating = pd.read_csv(title_ratings_url,sep='\t', low_memory=False)

In [19]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
rating.to_csv("Data/title_rating.csv.gz",compression='gzip',index=False)

In [20]:
akas.replace({'\\N':np.nan}, inplace = True)
basics.replace({'\\N':np.nan}, inplace = True)
rating.replace({'\\N':np.nan}, inplace = True)

In [21]:
#Exclude any movie with missing values for genre or runtime
basics = basics.dropna(subset = ['runtimeMinutes', 'genres'])

In [22]:
basics['titleType'].value_counts()

tvEpisode       1064241
short            574550
movie            367163
video            175133
tvMovie           88323
tvSeries          86592
tvSpecial         16470
tvMiniSeries      16082
tvShort            9380
videoGame           298
Name: titleType, dtype: int64

In [23]:
#Include only full-length movies (titleType = "movie").
basics = basics[basics.titleType == 'movie']

In [24]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1172,tt0001184,movie,Don Juan de Serrallonga,Don Juan de Serrallonga,0,1910,NaN,58,"Adventure,Drama"


In [25]:
#Include only fictional movies (not from documentary genre)
basics['genres'].value_counts()

Drama                           67929
Documentary                     62239
Comedy                          28910
Comedy,Drama                    10397
Drama,Romance                    9787
                                ...  
Adult,Drama,Musical                 1
Crime,Fantasy,Romance               1
Adult,Drama,War                     1
Documentary,Musical,Thriller        1
Crime,Fantasy,Sci-Fi                1
Name: genres, Length: 1383, dtype: int64

In [26]:
is_doc = basics['genres'].str.contains('documentary' , case=False)
basics = basics[~is_doc]

In [27]:
basics['genres'].value_counts()

Drama                        67929
Comedy                       28910
Comedy,Drama                 10397
Drama,Romance                 9787
Horror                        7360
                             ...  
Comedy,Reality-TV,Romance        1
Biography,Music,Mystery          1
Adventure,Horror,Musical         1
Music,Romance,Sci-Fi             1
Crime,Fantasy,Sci-Fi             1
Name: genres, Length: 1155, dtype: int64

In [29]:
#Include only movies that were released 2000 - 2021 (include 2000 and 2021)
basics = basics[(basics['startYear'] >= '2000') & (basics['startYear'] <= '2021')]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61090,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67636,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77930,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86767,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [30]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [31]:
#keep only US entries.
akas['region'].value_counts()

FR    3943552
JP    3943487
DE    3926631
IN    3871875
ES    3866627
       ...   
TV          1
NU          1
PW          1
NR          1
TC          1
Name: region, Length: 247, dtype: int64

In [32]:
akas = akas[akas.region == 'US']
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [33]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34790       True
61090       True
67636       True
77930      False
86767       True
           ...  
9188063     True
9188072     True
9188111    False
9188156     True
9188240    False
Name: tconst, Length: 136792, dtype: bool

In [34]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61090,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67636,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86767,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
92732,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy
...,...,...,...,...,...,...,...,...,...
9187527,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9187923,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9188063,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9188072,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [35]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79646 entries, 34790 to 9188156
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          79646 non-null  object
 1   titleType       79646 non-null  object
 2   primaryTitle    79646 non-null  object
 3   originalTitle   79646 non-null  object
 4   isAdult         79646 non-null  object
 5   startYear       79646 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  79646 non-null  object
 8   genres          79646 non-null  object
dtypes: object(9)
memory usage: 6.1+ MB


In [36]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1345187 entries, 5 to 33026685
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1345187 non-null  object
 1   ordering         1345187 non-null  int64 
 2   title            1345187 non-null  object
 3   region           1345187 non-null  object
 4   language         3694 non-null     object
 5   types            963721 non-null   object
 6   attributes       44828 non-null    object
 7   isOriginalTitle  1343812 non-null  object
dtypes: int64(1), object(7)
memory usage: 92.4+ MB


In [37]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257800 entries, 0 to 1257799
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1257800 non-null  object 
 1   averageRating  1257800 non-null  float64
 2   numVotes       1257800 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.8+ MB


In [38]:
with open('/Users/Yeajoon Yoo/codingdojo/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [39]:
tmdb.API_KEY =  login['api-key']

In [40]:
def get_movie_and_rating(movie_id):    
    
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()

    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [41]:
#Avengers
test = get_movie_and_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 228.284,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [42]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_rating.csv.gz']

In [43]:
YEARS_TO_GET =[2000,2001]

In [44]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [45]:
def read_and_fix_json(JSON_FILE):
    """Attempts to read in json file of records and fixes the final character
    to end with a ] if it errors.
    
    Args:
        JSON_FILE (str): filepath of JSON file
        
    Returns:
        DataFrame: the corrected data from the bad json file
    """
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    ## If read_json throws an error
    except:
        
        ## manually open the json file
        with open(JSON_FILE,'r+') as f:
            ## Read in the file as a STRING
            bad_json = f.read()
            
            ## if the final character doesn't match first, select the right bracket
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            ## Select expected final brakcet
            final_char = final_brackets[first_char]
            
            ## if the last character in file doen't match the first char, add it
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            ## Rewind to start of file and write new good_json to disk
            f.seek(0)
            f.write(good_json)
           
        ## Load the json file again now that its fixed
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

In [47]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
            
            #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc=f'Movies from {YEAR}',
                                      position=1,
                                      leave=True):
            # Attempt to retrieve then data for the movie id
            try:
                temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
                # Append/extend results to existing file using a pre-made function
                write_json(temp,JSON_FILE)
                # Short 20 ms sleep to prevent overwhelming server
                time.sleep(0.02)

            # If it fails,  make a dict with just the id and None for certification.
            except Exception as e:
                continue

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000: 0it [00:00, ?it/s]

Movies from 2001: 0it [00:00, ?it/s]